In [0]:
%sql
%sql
-- Create SQL Procedure for sp_ADS_SMA_CPP_MAIN_PREPARE ETL Process
CREATE OR REPLACE PROCEDURE gap_catalog.ads_etl_owner.sp_ADS_SMA_CPP_MAIN_PREPARE_PART2(
    p_load_date STRING,
    p_process_key STRING
)
LANGUAGE SQL
 SQL SECURITY INVOKER
COMMENT 'ETL procedure for sp_ADS_SMA_CPP_MAIN_PREPARE'
AS
BEGIN
    -- Declare local variables
    DECLARE my_log_id STRING;
    BEGIN
        CALL gap_catalog.log.start_performance_test(
            test_case_name => 'ADS_RDS_SMA',
            step_name => 'ADS_SMA_CPP_MAIN_PREPARE_PART1',
            psource_sys_origin  => 'MAIN',
            process_key => p_process_key,
            target_table => 'STG_SMA_CASE_PHASE_PROPERTIES_PARSE',
            log_id => my_log_id,
            p_load_date => p_load_date
        );

        TRUNCATE TABLE gap_catalog.ads_owner.STG_SMA_CASE_PHASE_PROPERTIES_PARSE;
        
        /* Fill Stage Table */
        insert into gap_catalog.ads_owner.STG_SMA_CASE_PHASE_PROPERTIES_PARSE 
SELECT 
    UPPER(attr_fp) || '.' || case_cidla AS cpprop_source_id,
    CASE 
        WHEN LENGTH(attr_fp) - LENGTH(REPLACE(attr_fp, '.', '')) > 0 
        THEN SUBSTRING(attr_fp, 1, LENGTH(attr_fp) - INSTR(REVERSE(attr_fp), '.')) || '.' || case_cidla
        ELSE NULL 
    END AS cpprop_parent_source_id,
    case_cidla AS case_source_id,
    UPPER(attr_fp) || '|' || casetype AS cpptp_source_id,
    UPPER(attr_fp) AS attr_fp,
    casetype,
    attr_value,
    attr_bvalue,
    attr_type,
    structuretype,
    mapping,
    timestamp
FROM (
    SELECT 
        att1.eventid,
        att1.case_cidla,
        att1.casetype,
        att1.timestamp,
        att1.attr_id,
        UPPER(att1.attr_fp) AS Attr_FP,
        UPPER(att1.attr_name) AS Attr_name,
        att1.attr_value,
        att1.attr_bvalue,
        att1.attr_type,
        att1.parentid,
        att1.modifiedby,
        att1.structuretype,
        att1.mapping
    FROM (
        SELECT 
            att0.eventid,
            att0.type,
            att0.case_cidla,
            att0.casetype,
            att0.timestamp,
            attr.id AS attr_id,
            attr.fullPath AS attr_fp,
            attr.name AS attr_name,
            attr.value AS attr_value,
            attr.blobValue AS attr_bvalue,
            attr.attributeType AS attr_type,
            attr.parentId AS parentid,
            attr.modifiedBy AS modifiedby,
            attr.structureType AS structuretype,
            attr.mapping AS mapping
        FROM (
            SELECT 
                j.eventid,
                j.cidla AS case_cidla,
                CASE 
                    WHEN j.casetype LIKE '% %' THEN 'ERR' 
                    ELSE j.casetype 
                END AS casetype,
                j.type,
                j.timestamp,
                jsondata
            FROM gap_catalog.ads_src_20250901.dlk_ads_sma_monitor_events_full j
            WHERE SYS_EFFECTIVE_DATE = CAST(p_load_date AS DATE) 
                AND j.type IN (
                    'CASE_CREATED',
                    'CASE_COMPLETED',
                    'CASE_WAITING',
                    'CASE_RUNNING',
                    'CASE_UPDATED',
                    'CASE_PHASE_CHANGED',
                    'CASE_RECONCILE'
                )
                AND FROM_UNIXTIME(
                    CAST(get_json_object(jsondata, '$.theCase.createdAt') AS BIGINT) / 1000
                ) >= ADD_MONTHS(DATE_TRUNC('day', CURRENT_DATE()), -25)
        ) att0
        LATERAL VIEW EXPLODE(
            FROM_JSON(
                get_json_object(jsondata, '$.theCase.attributes'),
                'array<struct<id:string,fullPath:string,name:string,value:string,blobValue:string,attributeType:string,parentId:string,modifiedBy:string,structureType:string,mapping:string>>'
            )
        ) attr_table AS attr
    ) att1
  );

        CALL gap_catalog.log.complete_performance_test(
            plog_id => my_log_id
        );

    END;
    
END;